In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from bayes_opt import BayesianOptimization, UtilityFunction
from matplotlib import gridspec
from matplotlib import cm
from PIL import Image
import io

def target(x, y):
    return np.exp(-(x - 2)**2 - (y - 2)**2) + np.exp(-(x - 6)**2 / 10 - (y - 6)**2 / 10) + 1 / (x**2 + y**2 + 1)

x = np.linspace(-2, 10, 100)
y = np.linspace(-2, 10, 100)
X, Y = np.meshgrid(x, y)
Z = target(X, Y)

optimizer = BayesianOptimization(
    f=target,  
    pbounds={'x': (-2, 10), 'y': (-2, 10)},
    random_state=42
)
acq_function = UtilityFunction(kind="ei", kappa=5)

optimizer.register({'x': 0, 'y': 0}, target(0, 0))
optimizer.register({'x': 4, 'y': 4}, target(4, 4))

def posterior(optimizer, x_obs, y_obs, grid):
    optimizer._gp.fit(x_obs, y_obs)
    mu, sigma = optimizer._gp.predict(grid, return_std=True)
    return mu, sigma

def plot_gp(optimizer, X, Y, Z, step=0):
    fig, axs = plt.subplots(2, 2, figsize=(18, 14))
    grid = np.c_[X.ravel(), Y.ravel()]
    
    x_obs = np.array([[res["params"]["x"], res["params"]["y"]] for res in optimizer.res])
    y_obs = np.array([res["target"] for res in optimizer.res])
    
    mu, sigma = posterior(optimizer, x_obs, y_obs, grid)
    mu = mu.reshape(X.shape)
    sigma = sigma.reshape(X.shape)

    c1 = axs[0, 0].contourf(X, Y, Z, cmap='viridis')
    axs[0, 0].scatter(x_obs[:, 0], x_obs[:, 1], color='grey', label='Observations')
    axs[0, 0].set_title("Target Function")
    fig.colorbar(c1, ax=axs[0, 0])

    c2 = axs[0, 1].contourf(X, Y, mu, cmap='plasma')
    axs[0, 1].scatter(x_obs[:, 0], x_obs[:, 1], color='grey', label='Observations')
    axs[0, 1].set_title(f"Gaussian Process Predicted Mean After {step} Steps")
    fig.colorbar(c2, ax=axs[0, 1])

    c3 = axs[1, 0].contourf(X, Y, sigma, cmap='coolwarm')
    axs[1, 0].scatter(x_obs[:, 0], x_obs[:, 1], color='grey', label='Observations')
    axs[1, 0].set_title(f"Gaussian Process Variance After {step} Steps")
    fig.colorbar(c3, ax=axs[1, 0])

    utility_function = UtilityFunction(kind="ei", kappa=5, xi=0)
    utility = utility_function.utility(grid, optimizer._gp, 0).reshape(X.shape)
    c4 = axs[1, 1].contourf(X, Y, utility, cmap='inferno')
    axs[1, 1].scatter(x_obs[:, 0], x_obs[:, 1], color='grey', label='Observations')
    axs[1, 1].set_title(f"Acquisition Function (EI) After {step} Steps")
    fig.colorbar(c4, ax=axs[1, 1])

    plt.tight_layout()
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.show()
    plt.close(fig)
    buf.seek(0)
    
    return buf

images = []
optimizer.maximize(init_points=2, n_iter=5)

for step in range(1, 6):
    optimizer.maximize(init_points=0, n_iter=1, acquisition_function=acq_function)
    buf = plot_gp(optimizer, X, Y, Z, step)
    image = Image.open(buf)
    images.append(image)

gif_path = 'BayesOptVisExample.gif'
images[0].save(
    gif_path, save_all=True, append_images=images[1:], duration=1000, loop=0
)